# 패키지 설치
pip 명령어로 의존성 있는 패키지를 설치합니다.


In [ ]:
!pip install ratsnlp

# 구글 드라이브 연동하기
모델 체크포인트 등을 저장해 둘 구글 드라이브를 연결합니다. 자신의 구글 계정에 적용됩니다.

In [2]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


# 각종 설정
모델 하이퍼파라메터(hyperparameter)와 저장 위치 등 설정 정보를 선언합니다.


In [3]:
from ratsnlp.nlpbook.generation import GenerationDeployArguments
args = GenerationDeployArguments(
    pretrained_model_name="skt/kogpt2-base-v2",
    downstream_model_dir="/gdrive/My Drive/nlpbook/checkpoint-generation",
)

downstream_model_checkpoint_fpath: /gdrive/My Drive/nlpbook/checkpoint-generation/epoch=1-val_loss=2.29.ckpt


# 모델 로딩
파인튜닝을 마친 GPT2 모델과 토크나이저를 읽어 들입니다.

In [ ]:
import torch
from transformers import GPT2Config, GPT2LMHeadModel
pretrained_model_config = GPT2Config.from_pretrained(
    args.pretrained_model_name,
)
model = GPT2LMHeadModel(pretrained_model_config)
fine_tuned_model_ckpt = torch.load(
    args.downstream_model_checkpoint_fpath,
    map_location=torch.device("cpu"),
)
model.load_state_dict({k.replace("model.", ""): v for k, v in fine_tuned_model_ckpt['state_dict'].items()})
model.eval()

In [5]:
from transformers import PreTrainedTokenizerFast
tokenizer = PreTrainedTokenizerFast.from_pretrained(
    args.pretrained_model_name,
    eos_token="</s>",
)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


# 인퍼런스 함수 선언
인퍼런스 함수를 선언합니다.

In [6]:
def inference_fn(
        prompt,
        min_length=10,
        max_length=20,
        top_p=1.0,
        top_k=50,
        repetition_penalty=1.0,
        no_repeat_ngram_size=0,
        temperature=1.0,
):
    try:
        input_ids = tokenizer.encode(prompt, return_tensors="pt")
        with torch.no_grad():
            generated_ids = model.generate(
                input_ids,
                do_sample=True,
                top_p=float(top_p),
                top_k=int(top_k),
                min_length=int(min_length),
                max_length=int(max_length),
                repetition_penalty=float(repetition_penalty),
                no_repeat_ngram_size=int(no_repeat_ngram_size),
                temperature=float(temperature),
           )
        generated_sentence = tokenizer.decode([el.item() for el in generated_ids[0]])
    except:
        generated_sentence = """처리 중 오류가 발생했습니다. <br>
            변수의 입력 범위를 확인하세요. <br><br> 
            min_length: 1 이상의 정수 <br>
            max_length: 1 이상의 정수 <br>
            top-p: 0 이상 1 이하의 실수 <br>
            top-k: 1 이상의 정수 <br>
            repetition_penalty: 1 이상의 실수 <br>
            no_repeat_ngram_size: 1 이상의 정수 <br>
            temperature: 0 이상의 실수
            """
    return {
        'result': generated_sentence,
    }

# 웹서비스 만들기 준비

`ngrok`은 코랩 로컬에서 실행 중인 웹서비스를 안전하게 외부에서 접근 가능하도록 해주는 도구입니다. `ngrok`을 실행하려면 [회원가입](https://dashboard.ngrok.com/signup) 후 [로그인](https://dashboard.ngrok.com/login)을 한 뒤 [이곳](https://dashboard.ngrok.com/get-started/your-authtoken)에 접속해 인증 토큰(authtoken)을 확인해야 합니다. 예를 들어 확인된 `authtoken`이 `test111`이라면 다음과 같이 실행합니다.

```bash
!mkdir /root/.ngrok2 && echo "authtoken: test111" > /root/.ngrok2/ngrok.yml
```

In [14]:
!mkdir /root/.ngrok2 && echo "authtoken: 2NG8D39Keyp1kkDOVg6Ni91rIg8_4oAFFG6XSpXDV3Hbc1rj5" > /root/.ngrok2/ngrok.yml

In [ ]:
!cat # 파일 탐색
!rm #파일 지우기
!rm -rf #폴더 지우기

# 웹서비스 개시
아래처럼 실행해 인퍼런스 함수를 웹서비스로 만듭니다.

In [16]:
from ratsnlp.nlpbook.generation import get_web_service_app
app = get_web_service_app(inference_fn)
app.run()

 * Serving Flask app 'ratsnlp.nlpbook.generation.deploy'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://adb3-34-125-161-71.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [20/Mar/2023 04:21:57] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Mar/2023 04:21:58] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [20/Mar/2023 04:24:01] "POST /api HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Mar/2023 04:24:12] "POST /api HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Mar/2023 04:24:28] "POST /api HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Mar/2023 04:24:42] "POST /api HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Mar/2023 04:24:53] "POST /api HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Mar/2023 04:25:01] "POST /api HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Mar/2023 04:25:22] "POST /api HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Mar/2023 04:25:46] "POST /api HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Mar/2023 04:25:56] "POST /api HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Mar/2023 04:26:09] "POST /api HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Mar/2023 04:26:24] "POST /api